# Đọc và lọc dữ liệu ranh giới tỉnh Yên Bái từ GeoJSON

Trong phần này, chúng ta sẽ sử dụng thư viện `json` để đọc file GeoJSON chứa ranh giới các tỉnh, sau đó lọc ra feature tương ứng với tỉnh Yên Bái dựa trên thuộc tính `ten_tinh`.

In [ ]:
import json

# Đường dẫn tới file geojson
geojson_path = 'mnt/data/diaphantinh.geojson'

with open(geojson_path, encoding='utf-8') as f:
    data = json.load(f)

# Lọc ra tỉnh Yên Bái
yen_bai = None
for feature in data['features']:
    if feature['properties']['ten_tinh'].lower() == 'yên bái':
        yen_bai = feature
        break

if yen_bai:
    print("Thông tin tỉnh Yên Bái:")
    print("Properties:", yen_bai['properties'])
    print("Geometry:", yen_bai['geometry'])
else:
    print("Không tìm thấy tỉnh Yên Bái")

# Vẽ bản đồ ranh giới tỉnh Yên Bái

Sau khi đã lọc được feature của tỉnh Yên Bái, chúng ta sẽ sử dụng GeoPandas và Matplotlib để đọc file GeoJSON và vẽ ranh giới tỉnh trên bản đồ.

In [ ]:
import json

yen_bai_geojson = {
    "type": "FeatureCollection",
    "features": [yen_bai]
}

with open("yen_bai.geojson", "w", encoding="utf-8") as f:
    json.dump(yen_bai_geojson, f, ensure_ascii=False, indent=2)

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Đọc file geojson vừa lưu
gdf = gpd.read_file("yen_bai.geojson")

# Vẽ bản đồ
gdf.plot(edgecolor='black', facecolor='lightblue', figsize=(8, 8))
plt.title("Ranh giới tỉnh Yên Bái")
plt.axis('equal')
plt.show()

# Đọc và lọc dữ liệu các huyện thuộc Yên Bái từ GADM

Tiếp theo, chúng ta sẽ đọc file GADM cấp huyện, lọc các feature có thuộc tính `NAME_1` là 'Yên Bái', và tạo GeoDataFrame cho các huyện này.

In [ ]:
import json
import geopandas as gpd

# Đọc file GeoJSON cấp huyện
with open("gadm41_VNM_3.json", encoding="utf-8") as f:
    gadm = json.load(f)

# Lọc các huyện thuộc Yên Bái (không có dấu cách)
yenbai_huyen = [
    feat for feat in gadm["features"]
    if feat["properties"].get("NAME_1", "").lower().replace(" ", "") == "yênbái"
]

print(f"Số huyện tìm được: {len(yenbai_huyen)}")  # Kiểm tra

# Tạo GeoDataFrame cho các huyện Yên Bái
gdf_huyen = gpd.GeoDataFrame.from_features(yenbai_huyen, crs="EPSG:4326")

# Vẽ bản đồ các huyện của tỉnh Yên Bái

Chúng ta sẽ vẽ bản đồ các huyện Yên Bái với màu sắc phân biệt từng huyện, đồng thời hiển thị tên huyện trên bản đồ để dễ dàng nhận diện.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 8))
gdf_huyen.plot(ax=ax, edgecolor='black', linewidth=1, alpha=0.6, column="NAME_2", cmap="tab20", legend=True)
for idx, row in gdf_huyen.iterrows():
    if row.geometry.centroid.is_empty:
        continue
    x, y = row.geometry.centroid.x, row.geometry.centroid.y
    plt.text(x, y, row["NAME_2"], fontsize=8, ha='center', va='center')
plt.title("Các huyện của tỉnh Yên Bái (GADM)")
plt.axis('equal')
plt.show()

# Tạo và vẽ lưới ô vuông lớn (10km x 10km) phủ tỉnh Yên Bái

Ở phần này, chúng ta sẽ tạo lưới các ô vuông lớn (10km x 10km) phủ toàn bộ tỉnh Yên Bái bằng shapely và geopandas. Thông tin các ô và tâm ô sẽ được lưu ra file CSV để sử dụng cho các bước tiếp theo.

In [ ]:
import geopandas as gpd
from shapely.geometry import box, Point
import numpy as np
import pandas as pd

# Đọc file geojson Yên Bái
gdf = gpd.read_file("yen_bai.geojson")
gdf_utm = gdf.to_crs(epsg=32648)  # UTM phù hợp miền Bắc VN

# Tạo lưới lớn 10km x 10km
minx, miny, maxx, maxy = gdf_utm.total_bounds
big_size = 10000
big_ids = []
centers_utm = []
centers_lon = []
centers_lat = []

i = 0
for x0 in np.arange(minx, maxx, big_size):
    for y0 in np.arange(miny, maxy, big_size):
        x1, y1 = x0 + big_size, y0 + big_size
        cell = box(x0, y0, x1, y1)
        if cell.intersects(gdf_utm.unary_union):
            centroid = cell.centroid
            big_ids.append(f"big_{i}")
            centers_utm.append(centroid)
            i += 1

# Chuyển các điểm trung tâm về WGS84
gdf_centers = gpd.GeoDataFrame({'big_square_id': big_ids, 'geometry': centers_utm}, crs=gdf_utm.crs)
gdf_centers_wgs = gdf_centers.to_crs(epsg=4326)  # WGS84

centers_lon = gdf_centers_wgs.geometry.x
centers_lat = gdf_centers_wgs.geometry.y

df = pd.DataFrame({
    "big_square_id": big_ids,
    "center_lon": centers_lon,
    "center_lat": centers_lat
})
df.to_csv("yen_bai_big_squares_centers_wgs84.csv", index=False)
print("Đã tạo file yen_bai_big_squares_centers_wgs84.csv")

# Vẽ bản đồ các điểm trung tâm ô lớn trên nền ranh giới tỉnh

Cuối cùng, chúng ta sẽ đọc file CSV các điểm trung tâm ô lớn và vẽ các điểm này trên bản đồ ranh giới tỉnh Yên Bái để trực quan hóa vị trí các ô vuông lớn đã tạo.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

# Đọc ranh giới Yên Bái
gdf = gpd.read_file("yen_bai.geojson")

# Đọc các điểm trung tâm ô lớn (WGS84) từ CSV
df = pd.read_csv("yen_bai_big_squares_centers_wgs84.csv")
centers = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.center_lon, df.center_lat),
    crs="EPSG:4326"
)

# Vẽ bản đồ
fig, ax = plt.subplots(figsize=(8, 8))
gdf.to_crs(epsg=4326).plot(ax=ax, edgecolor='black', facecolor='none')
centers.plot(ax=ax, color='red', markersize=30)
plt.title("Ranh giới tỉnh Yên Bái và các điểm trung tâm ô lớn")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()